In [ ]:
#FA'IQ ALI SUTIONO -INFORMATIKA A
#1152100040
import pandas as pd
import numpy as np
from scipy.stats import chisquare
from scipy.stats.distributions import chi2
from scipy import stats
import scipy
from decimal import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from google.colab import files
from scipy.stats import chi2_contingency

uploaded = files.upload()
# Membaca file csv
hd = pd.read_csv("heart.csv")
# memisahkan field yang akan dicari korelasinya
column = ["age", "cp", "trestbps", "chol", "fbs", "restecg",
          "thalach", "exang", "oldpeak", "slope", "ca", "thal"]
qualified = ["sex"]
# Cek jika ada data yang Null
cek = hd.isnull().sum().sum()
# memisahkan target dengan variabel lainnya (non-shuffle)
feature = hd[["age", "sex", "cp", "trestbps", "chol", "fbs",
             "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
target = hd["target"]
# memisahkan target dengan variabel lainnya (shuffle)
Shuffled_data = shuffle(hd, random_state=7)
feature_shuffle = Shuffled_data[["age", "sex", "cp", "trestbps", "chol", "fbs",
                                 "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
target_shuffle = Shuffled_data["target"]


# Membuat fungsi chisquare dengan fitur otomatis "Feature Selection" dan kesimpulan

# fungsi dibawah ini adalah optional
def chisFunc(table, col):
    # Membuat tabel contingen dengan crosstab antara sex sebagai row dan lainnya sebagi column (loop)
    df = pd.crosstab(table["sex"], table[col], margins=True)
    print(df)
    # Fungsi dari library untuk mengetahui nilai chisquare, derajat kebebasan, dan p-value
    stat = stats.chi2_contingency(df)
    stati, p, dof, expected = chi2_contingency(df)
    #format output variabel stat: [Chisquare value, p-value, Degree of Freedom, Expected Value(array)]
    print(f'Degree of Freedom: {stat[2]}')
    print(f'ChiSquare Value: {stat[0]}')
    # Fungsi untuk mengetahui critical value dengan alpha =0,05 dan derajat kebebasan = stat[2](berbeda2)
    critVal = scipy.stats.chi2.ppf(1-.05, df=stat[2])
    # Jika p-value kurang dari 0,05 dan total chisquare > critical value maka
    if stat[0] > critVal:
        # memisahkan variabel yang memiliki korelasi yang kuat dan dimasukan ke array qualified
        qualified.append(col)
        print(
            f"Chi-Square value = {stat[0]} > Critical Value = {critVal} sehingga\nHipotesa 0 ditolak dan Hipotesa Ha diterima yang artinya ada hubungan yang kuat antara sex dengan {col}\n\n")
    else:
        print(
            f"Chi-Square value = {stat[0]} < Critical Value  = {critVal} sehingga\nHipotesa 0 diterima dan Hipotesa Ha ditolak yang artinya hubungan lemah antara sex dengan {col}\n\n")


# membuat tabel baru berdasarkan korelasi yang kuat antara "sex" dengan yang lain
choosen_df = pd.DataFrame(hd, columns=qualified)

# membuat fungsi naive bayes


def naiveBayes(fea, tar):
    # memisahkan data train dan data test
    X_train, X_test, y_train, y_test = train_test_split(
        fea, tar, random_state=7, test_size=.50)
    # Mengambil data test dari data train
    # Deklarasi Method Naive Bayes
    naivebayes = GaussianNB()
    # memasukan data train pada parameter x dan y dengan memanggil method dari GaussianNB()
    trained = naivebayes.fit(X_train, y_train)
    # memprediksi hasil train dari variabel trained dalam bentuk array
    prediction = trained.predict(X_test)
    # array data yang sebenarnya yang akan digunakan untuk membandingkan prediction
    dataFact = np.array(y_test)
    # tingkat accuracy yang diperoleh
    score = accuracy_score(y_test, prediction, sample_weight=None)
    print(f"data prediksi:\n {prediction}")
    print(f"data faktual:\n {dataFact}")
    print(f"score: {score}")

# membuat fungsi knn


def knn(fea, tar):
    # memisahkan data train dengan data test
    X_train, X_test, y_train, y_test = train_test_split(
        fea, tar, test_size=0.50, random_state=7)
    # deklarasi dari mana k akan dimulai
    start_nn = 1
    # deklarasi sampai mana k akan selesai
    end_nn = 7
    # array untuk menyimpan setiap k yang di loop
    nn = []
    # array untuk menyimpan setiap score yang digenerate
    scores = []
    # looping untuk menghasilkan nilai k(ganjil)
    for i in range(start_nn, end_nn+1):
        if i % 2 != 0:
            nn.append(i)
    # looping untuk mengeluarkan nilai j dalam nn dimana j adalah nilai K dari hasil looping diatas yang akan digunkan untuk parameter KneighborsClassifier
    for j in nn:
        # deklarasi method dari library sklearn
        classifier = KNeighborsClassifier(
            n_neighbors=j, metric='minkowski', p=2)
        # mengisi parameter fit dengan fitur dan label untuk klasifikasi
        classifier.fit(X_train, y_train.values.ravel())
        # memprediksi data test x berdasarkan hasil klasifikasi diatas
        Pred = classifier.predict(X_test)
        # menambahkan tiap score ke dalam array scores
        scores.append(accuracy_score(y_test, Pred))
        # code dibawah ini akan mendeteksi hasil knn yang terbesar untuk digunakan sebagai pembanding dengan metode lain (naive bayes)
    print(f"{scores}\nMengeluarkan nilai terbesar untuk dibandingkan...")
    # mencari value terbesar di array scores
    highest_val = max(scores)
    print(
        f'nilai terbesar dari K 1-7 adalah {highest_val} dengan K = {nn[scores.index(highest_val)]}')


print(f"Naive Bayes non shuffle:\n")
naiveBayes(feature, target)
print("")
print(f"Naive Bayes shuffle:\n ")
naiveBayes(feature_shuffle, target_shuffle)
print("")
print(f"KNN non shuffle:\n")
knn(feature, target)
print("")
print(f"KNN shuffle:\n ")
knn(feature_shuffle, target_shuffle)
print("")


Saving heart.csv to heart (14).csv
Naive Bayes non shuffle:

data prediksi:
 [1 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1
 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0
 1 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1
 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1
 1 0 1 1]
data faktual:
 [1 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1
 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0 1 0
 1 0 1 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1
 0 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 0 1 0 1
 1 0 1 1]
score: 0.7763157894736842

Naive Bayes shuffle:
 
data prediksi:
 [0 0 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0
 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0

In [ ]:
# looping kolom untuk fungsi chisFunc
for i in column:
    # parameter diisi dengan nama tabel dan setiap kolom dari array column
    chisFunc(hd, i)

age  29  34  35  37  38  39  40  41  42  43  ...  66  67  68  69  70  71  74  \
sex                                          ...                               
0     0   1   1   1   0   2   0   4   2   2  ...   3   3   1   1   0   3   1   
1     1   1   3   1   3   2   3   6   6   6  ...   4   6   3   2   4   0   0   
All   1   2   4   2   3   4   3  10   8   8  ...   7   9   4   3   4   3   1   

age  76  77  All  
sex               
0     1   0   96  
1     0   1  207  
All   1   1  303  

[3 rows x 42 columns]
Degree of Freedom: 82
ChiSquare Value: 42.579366296088295
nChi-Square value = 42.579366296088295 < Critical Value  = 104.13873823027387 sehingga
Hipotesa 0 diterima dan Hipotesa Ha ditolak yang artinya hubungan lemah antara sex dengan age


cp     0   1   2   3  All
sex                      
0     39  18  35   4   96
1    104  32  52  19  207
All  143  50  87  23  303
Degree of Freedom: 8
ChiSquare Value: 6.822075813213287
nChi-Square value = 6.822075813213287 < Critical Value